<a href="https://colab.research.google.com/github/Jaesu26/Dacon-Basic/blob/main/%EB%89%B4%EC%8A%A4_%EA%B7%B8%EB%A3%B9_%EB%B6%84%EB%A5%98_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 뉴스 그룹 분류 경진대회

## 패키지 import 및 데이터 전처리

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from os.path import join
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from glob import glob
# nltk.download('all')

In [61]:
SEED = 22
N_FOLD = 5
SAVE_PATH = './weight'
LEARNING_RATE = 0.001
EPOCHS = 32
BATCH_SIZE = 64

In [4]:
from google.colab import drive

drive.mount('/content/drive') ## 구글코랩과 구글드라이브 연결

Mounted at /content/drive


In [7]:
%cd '/content/drive/MyDrive/Github/Dacon-Basic/뉴스-그룹-분류/뉴스-그룹-모델링'

/content/drive/MyDrive/Github/Dacon-Basic/뉴스-그룹-분류/뉴스-그룹-모델링


In [6]:
#!unzip -o '뉴스그룹분류경진대회.zip' ## 현재 디렉토리에 존재하는 zip파일을 현재 디렉토리에 압축해제

In [9]:
df = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')
submission = pd.read_csv('../Data/sample_submission.csv')

In [10]:
def remove_stopwords(df_text, stopwords_list):
    df_words = df_text.split()
    df_text_without_stopwords  = ' '.join([df_word for df_word in df_words if df_word not in stopwords_list])
    return df_text_without_stopwords

In [11]:
stopwords_list = stopwords.words('english') ## nltk에서 제공하는 불용어사전 이용

In [12]:
df['text_without_stopwords'] = df['text'].apply(lambda text: remove_stopwords(text, stopwords_list))
test['text_without_stopwords'] = test['text'].apply(lambda text: remove_stopwords(text, stopwords_list))

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))

In [47]:
tfidf.fit(df['text_without_stopwords'])

TfidfVectorizer()

In [48]:
train_tfidf = tfidf.transform(df['text_without_stopwords'])
test_tfidf = tfidf.transform(test['text_without_stopwords'])

In [17]:
target = df['target']

## 데이터 셋 및 딥러닝 모델 정의

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import gc

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
print(f'현재 device는 {device}입니다')

현재 device는 cuda입니다


In [23]:
def seed_everything(seed: int = 22):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [24]:
seed_everything(SEED) ## 재현을 위한 seed 고정

In [25]:
class CustomDataset(Dataset):
  
    def __init__(self, csr_matrix, labels=None, transform=None, target_transform=None):
        self.X = csr_matrix  ## 희소행렬
        self.labels = labels  ## 라벨
        self.transform = transform  ## X 변환방법
        self.target_transform = target_transform ## 라벨 변환방법
    
    def __getitem__(self, idx):
        data = self.X[idx]
        if self.transform is not None: 
            data = self.transform(data) 
            
        if self.labels is not None:
            label = self.labels[idx]
            if self.target_transform:
                label = self.target_transform(label)             
            return data, label  ## train   
        
        return data  ## test는 라벨이 없다
    
    def __len__(self):
        return self.X.shape[0] ## 데이터 개수 반환

In [76]:
## 커스텀 train 데이터 로더를 위한 함수
def collate_batch_train(batch):
    """희소행렬이 input으로 들어오면 batch에 해당하는 부분만 텐서로 변환하여 리턴"""
    text_list = []
    label_list = []
    for text_tfidf, text_class in batch:
        text_list.append(text_tfidf.toarray())
        label_list.append(text_class)

    label_tensor = torch.tensor(np.array(label_list), dtype=torch.int64).squeeze() ## (N, 1, P) shape -> (N, P) shape
    text_tensor = torch.tensor(np.array(text_list), dtype=torch.float32).squeeze() ## (N, 1, P) shape -> (N, P) shape
    return text_tensor, label_tensor

In [75]:
## 커스텀 test 데이터 로더를 위한 함수
def collate_batch_test(batch):
    """희소행렬이 input으로 들어오면 batch에 해당하는 부분만 텐서로 변환하여 리턴"""
    text_list = []
    for text_tfidf in batch:
        text_list.append(text_tfidf.toarray())

    text_tensor = torch.tensor(np.array(text_list), dtype=torch.float32).squeeze() ## (N, 1, P) shape -> (N, P) shape
    return text_tensor

`-` 데이터 셋의 아웃풋이 데이터 로더의 인풋이다

`-` 그리고 데이터 로더의 아웃풋을 조정하는 DataLoader의 메소드가 `collate_fn`이다

In [27]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_model = nn.Sequential(
            nn.Linear(train_tfidf.shape[1], 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 20) ## softmax는 옵티마이저(CrossEntorpyLoss)에서 수행
        )
        
    def forward(self, x):
        x = self.linear_model(x) 
        return x

In [28]:
## Linear layer 가중치 초기화
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        y = m.in_features
        m.weight.data.normal_(0.0, 1/np.sqrt(y))
        m.bias.data.fill_(0)

In [29]:
## accuracy 계산
def accuracy(true, pred):
    return sum(true == pred) / len(true)

In [63]:
class EarlyStopping:
    ## 코드 참고: https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
    
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='./weight', n_fold=0):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc = None
        self.delta = delta
        self.path = path
        self.n_fold = n_fold

    def __call__(self, model, val_loss, val_acc):

        score = -val_loss ## val_loss는 작을수록 좋다 ## score는 0에 가까울수록 좋다

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, val_acc, model)  
        elif score < self.best_score + self.delta: ## loss가 개선되지 않았을 때
            self.counter += 1 ## 카운팅 +1
            # print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience: ## 만약 loss가 개선되지 않은 스탭이 patience보다 크거나 같아진다면 조기중단
                self.early_stop = True
        else: ## loss가 개선됨
            self.best_score = score ## score 갱신
            self.save_checkpoint(val_loss, val_acc, model) ## loss와 model 저장
            self.counter = 0 ## loss가 개선되었으므로 0으로 초기화

    def save_checkpoint(self, val_loss, val_acc, model):
        """validation loss가 감소하면 모델을 저장"""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.5f} -> {val_loss:.5f})  Saving model ...')
        torch.save(model.state_dict(), self.path + f'/best_{self.n_fold}.pt') ## 모델의 계층별 가중치를 지정한 경로에 저장
        self.val_loss_min = val_loss ## 모델이 더 좋게 갱신되었으므로 이때의 valid loss를 기준치로 변경
        self.val_acc = val_acc ## 이때의 valid accuracy도 변경해준다

## 모델 교차검증

In [31]:
skfold = StratifiedKFold(n_splits=N_FOLD, random_state=SEED, shuffle=True) ## k겹 교차검증

In [32]:
loss_fn = torch.nn.CrossEntropyLoss()   ## 손실 함수에 소프트맥스 함수 포함 -> net 내부에서 마지막 활성화함수로 소프트맥스 사용안해도 됨

In [33]:
def train(model: nn.Module, dataloader, optimizer, scheduler, loss_fn):
    """학습된 모델과 평균 훈련 오차를 리턴"""
    model.train() ## 훈련모드
    train_avg_loss = 0 ## 에폭별 배치단위 평균 훈련 오차
    train_total_batch = len(dataloader) ## 배치 크기

    for X, y in dataloader: ## 미니 배치 단위로 꺼내온다, X는 미니 배치, y는 레이블
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad() ## 그래디언트 초기화
        yhat = model(X) ## y_hat을 구한다
        loss = loss_fn(yhat, y).to(device) ## 오차를 계산 ## train loss
        loss.backward()  ## 미분
        optimizer.step() ## 업데이트
        train_avg_loss += (loss.item() / train_total_batch) ## 각 배치마다 훈련 오차 누적

    ## epoch마다 학습률 조절
    scheduler.step()

    return model, train_avg_loss

In [62]:
def evaluate(model: nn.Module, dataloader, loss_fn):
    """모델에 대한 평균 평가 오차와 평가 정확도를 리턴"""
    valid_avg_acc, valid_avg_loss = 0, 0

    model.eval() ## 평가모드
    with torch.no_grad(): ## 평가할 땐 역전파를 쓸 필요가 없으니까
        for X, y in dataloader: 
            X, y = X.to(device), y.to(device)
            yhat = model(X)
            loss = loss_fn(yhat, y) ## valid loss
            acc = accuracy(y.cpu().data.numpy(), yhat.cpu().data.numpy().argmax(-1))       
            valid_avg_acc += (acc * len(y) / len(dataloader.dataset)) ## 각 배치마다 정확도(정답 개수 / 전체 개수)
            valid_avg_loss += loss.item() / len(dataloader) ## 각 배치마다 평가 오차 누적    

    return valid_avg_loss, valid_avg_acc

In [64]:
net_acc = [] ## fold별 valid셋의 평균 정확도
net_loss = [] ## fold별 valid셋의 평균 손실

for i, (train_idx, valid_idx) in enumerate(skfold.split(df, target)):
    gc.collect()
    torch.cuda.empty_cache()
    print(f'{i + 1} Fold Training......')
    X_train, X_valid = train_tfidf[train_idx], train_tfidf[valid_idx] 
    y_train, y_valid = target.iloc[train_idx], target.iloc[valid_idx]
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.int64) ## target을 텐서로 변환
    y_valid = torch.tensor(y_valid.to_numpy(), dtype=torch.int64) ## target을 텐서로 변환
    
    ## early stopping
    early_stopping = EarlyStopping(patience=7,
                                     verbose=False,
                                     path=SAVE_PATH,
                                     n_fold=i+1) ## 특정 횟수 에폭후에도 valid loss가 작아지지 않으면 조기 중단
    
    ## Linear 모델
    net = Net().to(device)
    net.apply(init_weights) ## Linear layer 가중치 초기화
    
    ## Dataset, Dataloader
    train_dataset = CustomDataset(X_train, y_train)
    valid_dataset = CustomDataset(X_valid, y_valid)
    
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=collate_batch_train)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_train)

    ## optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE) ## 옵티마이저에 최적화할 파라미터와 학습률 전달
    
    ## scheduler
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                                 lr_lambda=lambda epoch: 0.95 ** epoch,
                                                 last_epoch=-1,
                                                 verbose=False)
    
    ## fold별로 모델 학습
    for epoch in tqdm(range(EPOCHS)): ## (배치사이즈 * 에폭)만큼 훈련시킴
        net, train_avg_loss = train(net, train_dataloader, optimizer, scheduler, loss_fn)  ## 모델 학습
        valid_avg_loss, valid_avg_acc = evaluate(net, valid_dataloader, loss_fn)  ## 모델 평가

        if epoch % 5 == 0 or epoch == EPOCHS - 1: 
            ## 5의 배수값을 가지는 에폭마다 평균 배치 훈련 오차와 평가 오차 출력
            print('[Epoch: {:>3}] train loss = {:>.5}  valid loss = {:>.5}'.format(epoch + 1, train_avg_loss, valid_avg_loss)) 
            
        ## epoch마다 early stopping 실행
        early_stopping(net, valid_avg_loss, valid_avg_acc) ## __call__ function
        if early_stopping.early_stop: ## early_stop이 true이면
            if epoch % 5 != 0 and epoch != EPOCHS - 1:
                print('[Epoch: {:>3}] train loss = {:>.5}  valid loss = {:>.5}'.format(epoch + 1, train_avg_loss, valid_avg_loss)) 
            print('Early stopping!')
            break 

    net_acc.append(early_stopping.val_acc) ## fold별 loss가 가장 작은 모델의 정확도
    net_loss.append(early_stopping.val_loss_min) ## fold별 loss가 가장 작은 모델의 손실
    
    ## fold별 평가 루프 종료시 가장 작은 loss와 이때의 accuracy를 출력
    print(f'{i + 1} Fold -> Best Valid Loss: {early_stopping.val_loss_min:.4f}  Best Valid Accuracy: {early_stopping.val_acc:.4f}\n\n')
    
## 마지막으로 폴드별 가장 loss가 작은 모델들의 평균 정확도와 평균 손실을 출력
print(f'{skfold.n_splits}Fold Mean Valid Accuracy: {np.mean(net_acc):.4f}')
print(f'{skfold.n_splits}Fold Mean Valid Loss: {np.mean(net_loss):.4f}')                                               

1 Fold Training......


  0%|          | 0/32 [00:00<?, ?it/s]

[Epoch:   1] train loss = 1.4542  valid loss = 0.9986


 19%|█▉        | 6/32 [01:10<04:57, 11.43s/it]

[Epoch:   6] train loss = 0.039587  valid loss = 0.91558


...

[Epoch:   1] train loss = 1.4615  valid loss = 1.0497


 19%|█▉        | 6/32 [01:06<04:49, 11.15s/it]

[Epoch:   6] train loss = 0.035129  valid loss = 1.003


 31%|███▏      | 10/32 [02:01<04:27, 12.15s/it]

[Epoch:  11] train loss = 0.033441  valid loss = 1.0212
Early stopping!
5 Fold -> Best Valid Loss: 0.9901  Best Valid Accuracy: 0.7313


5Fold Mean Valid Accuracy: 0.7405
5Fold Mean Valid Loss: 0.9352


## test 예측

`-` softmax function을 취하면 20개의 원소 중 최대값의 인덱스 번호가 최종 예측값이 된다

In [135]:
def predict(model: nn.Module, dataloader, save_path):
    model = model.to(device)
    weigh_path_list = glob(save_path + '/*.pt')
    test_probs = np.zeros(shape=(len(dataloader.dataset), 20)) ## test예측값

    for weight in weigh_path_list :
        model.load_state_dict(torch.load(weight))
        model.eval()
        probs = None
        
        with torch.no_grad(): 
            for test_batch in dataloader:
                test_batch = test_batch.to(device)
                outputs = model(test_batch).cpu().numpy()
                if probs is None:
                    probs = outputs
                else:
                    probs = np.concatenate([probs, outputs])

        test_probs += (probs / N_FOLD)

    _, test_preds = torch.max(torch.tensor(test_probs), dim=1) ## 최대값과 인덱스를 반환
    ## return (value, indices), 1차원을 기준으로 max를 구한다
    ## 1차원을 기준으로 max를 구하므로 1차원을 없앤 9233개의 max가 반환된다
    ## 행별로 20개의 값 중 최대값을 리턴(총 9233개)
    ## 만약 dim=0으로 했다면 1번째 열~20번째 열별로 최대값을 구하므로 총 20개의 max값이 리턴됨

    return test_preds  

In [106]:
net = Net().to(device)

In [107]:
test_dataset = CustomDataset(test_tfidf)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch_test)

In [136]:
pred = predict(net, test_dataloader, SAVE_PATH)

In [137]:
submission['target'] = pred
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,7


In [139]:
submission.to_csv('../Data/submission_DNN.csv', index=False)